In [ ]:
import cirq
import numpy as np

# Function to create an oracle gate from a given unitary matrix
def create_oracle_gate(oracle_matrix):
    return cirq.MatrixGate(oracle_matrix)

# Function to prompt the user to input a 4x4 unitary matrix
def input_unitary_matrix():
    print("Enter the 64 comma-separated values for the 8x8 unitary matrix:")
    #matrix_values = [input0,()].strip().split(',')
    matrix_values = "00,01,10,11".strip().split(',')
    #oracle_matrix = np.array([[float(matrix_values[i]) for i in range(8)],
    #                          [float(matrix_values[i]) for i in range(8, 16)],
    #                          [float(matrix_values[i]) for i in range(16, 24)],
    #                          [float(matrix_values[i]) for i in range(24, 32)],
    #                          [float(matrix_values[i]) for i in range(32, 40)],
    #                          [float(matrix_values[i]) for i in range(40, 48)],
    #                          [float(matrix_values[i]) for i in range(48, 56)],
    #                          [float(matrix_values[i]) for i in range(56, 64)]])
    oracle_matrix = np.array([[float(matrix_values[i]) for i in range(2)],
                              [float(matrix_values[i]) for i in range(2, 4)],
                              [float(matrix_values[i]) for i in range(4, 6)],
                              [float(matrix_values[i]) for i in range(6, 8)]])
    return oracle_matrix
    
def determine_function_type(circuit_result):
    # Get the measurement outcomes of the initial two qubits
    measurement_outcomes = [circuit_result.measurements[f'result_{i}'][0][0] for i in range(2)]

    # Determine the type of function based on the measurement outcomes
    if all(outcome == 0 for outcome in measurement_outcomes):
        return "constant"
    else:
        return "balanced"
        
# Define Deutsch's algorithm
def deutsch_algorithm(oracle_gate):
    # Define qubits
    qubits = cirq.LineQubit.range(3)

    # Create a quantum circuit
    circuit = cirq.Circuit()

   
# Apply X gate (NOT gate) on the third qubit
    circuit.append(cirq.X(qubits[-1]))

# Apply Hadamard gate to all qubits
    circuit.append([cirq.H(q) for q in qubits])

# Apply the matrix gate on all inputs
    circuit.append(oracle_gate(*qubits))

# Apply Hadamard gate to all outputs except the last qubit
    for qubit in qubits[:-1]:
        circuit.append(cirq.H(qubit))


   # Measure the initial two qubits
    circuit.append([cirq.measure(q, key=f'result_{i}') for i, q in enumerate(qubits[:2])])

    return circuit

# Prompt the user to input the 4x4 unitary matrix
oracle_matrix = input_unitary_matrix()

# Create the oracle gate from the user-defined unitary matrix
oracle_gate = create_oracle_gate(oracle_matrix)

# Run Deutsch's algorithm with the user-defined oracle gate
circuit = deutsch_algorithm(oracle_gate)

# Simulate the circuit 5 times and collect results
simulator = cirq.Simulator()
measurement_outcomes_list = []

for _ in range(10):
    result = simulator.run(circuit)
    measurement_outcomes = [result.measurements[f'result_{i}'][0][0] for i in range(2)]
    measurement_outcomes_list.append(measurement_outcomes)
    
# Print all measurement outcomes
print("Measurement outcomes:", measurement_outcomes_list)  

# Determine the type of function based on the measurement outcomes
function_type = determine_function_type(result)
print("Function type:", function_type)

# Print the circuit
print("\nCircuit:")
print(circuit)



Using a default 8x8 balanced oracle (XOR).
Measurement outcomes: [[np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)], [np.int8(1), np.int8(1)]]
Function type: balanced

Circuit:
              ┌                       ┐
              │1. 0. 0. 0. 0. 0. 0. 0.│
              │0. 1. 0. 0. 0. 0. 0. 0.│
              │0. 0. 0. 1. 0. 0. 0. 0.│
0: ───H───────│0. 0. 1. 0. 0. 0. 0. 0.│───H───M('result_0')───
              │0. 0. 0. 0. 0. 1. 0. 0.│
              │0. 0. 0. 0. 1. 0. 0. 0.│
              │0. 0. 0. 0. 0. 0. 1. 0.│
              │0. 0. 0. 0. 0. 0. 0. 1.│
              └                       ┘
              │
1: ───H───────#2──────────────────────────H───M('result_1')───
              │
2: ───X───H───#3──────────────────────────────────────────────
